# pymongo基本操作

### 创建数据库

In [ ]:
import pymongo

# 仅仅创建mongodb连接,但未验证,没有连接到具体的数据库,相当于打开一个shell
client = pymongo.MongoClient(host='127.0.0.1', port=27017)
# 创建数据库
db = pymongo.database.Database(client, 'testdb')
db.command("updateUser", "arvin", pwd="arvin123")

# 创建管理员用户,密码和权限
db.command("createUser", "admin", pwd="admin@123456.", roles=["root"])
# 创建普通用户,密码和读取权限
db.command("createUser", "arvin", pwd="arvin@123456.", roles=["readWrite"])

### 检测mongodb服务是否可用

In [42]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

client = MongoClient()

try:
    # The ismaster command is cheap and does not require auth.
    client.admin.command('ismaster')
    print('Server is available')
except ConnectionFailure:
    print("Server is not available")

Server is available


### 连接mongodb(无auth验证)

无验证式连接数据库通常是在第一次连接数据库时使用到,此时没有建立数据库,没有配置用户角色和读写权限.

In [26]:
import pymongo

# 仅仅创建mongodb连接,但未验证,没有连接到具体的数据库,相当于打开一个shell
client = pymongo.MongoClient(host='127.0.0.1', port=27017)
# 创建数据库
db = pymongo.database.Database(client, 'testdb')
db.command("updateUser", "arvin", pwd="arvin123")

# 创建管理员用户,密码和权限
db.command("createUser", "admin", pwd="admin@123456.", roles=["root"])
# 创建普通用户,密码和读取权限
db.command("createUser", "arvin", pwd="arvin@123456.", roles=["readWrite"])
# 创建普通用户,密码和只读权限
db.command("createUser", "xiaoming", pwd="xiaoming@123456.", roles=["read"])
# 创建普通用户,密码和只写权限
db.command("createUser", "xiaohong", pwd="xiaohong@123456.", roles=["write"])
# 修改用户密码
db.command("updateUser", "xiaoming", pwd="xiaoming@123.")
# 修改用户读取权限
db.command("updateUser", "xiaoming", roles=["readWrite"])

{'ok': 1.0}

### mongodb连接(带有auth验证)

pymongo是线程安全的,但是不是fork-safe. 当多进程使用同一个client实例会报错:

**解决办法:** 在连接mongo数据库时,设置关键字`connect=False`,即在mongodb实例化时不建立连接,等到有数据库操作才进行数据库连接.

In [ ]:
import pymongo

try:
    # Python 3.x
    from urllib.parse import quote_plus
except ImportError:
    # Python 2.x
    from urllib import quote_plus

MONGODB = {
    "user": "arvin",
    "passwd": "arvin@123456.",
    "host": "127.0.0.1:27017",
    "dbname": "testdb"
}

# 方式一
# 以类的方式创建mongodb的实例
class ConnectMongo(object):
    
        def __init__(self, MONGODB):
        self.client = MongoClient(
            'mongodb://{}:{}@{}/{}'.format(MONGODB['user'],
                                           MONGODB['passwd'],
                                           MONGODB['host'],
                                           MONGODB['dbname']),
            connect=False
        )
        self.mongodb = self.client[MONGODB['dbname']]

    def __del__(self):
        self.client.close()
        
# 方式二        
def connect_mongo(MONGODB):
    client = MongoClient(
        'mongodb://{}:{}@{}/{}'.format(MONGODB['user'],
                                       MONGODB['passwd'],
                                       MONGODB['host'],
                                       MONGODB['dbname']),
        connect=False)
    mongodb = client[MONGODB['dbname']]
    return mongodb

# 方式三: 处理特殊字符 
def connect_mongo(mongo_param):
    uri =  'mongodb://{}:{}@{}/{}'.format(quote_plus(mongo_param['user']),
                                          quote_plus(mongo_param['passwd']),
                                          mongo_param['host'],
                                          mongo_param['dbname'])
    try:
        client = pymongo.MongoClient(uri, connect=False)
        mongodb = client[mongo_param['dbname']]
        return mongodb
    except Exception as e:
        print(e)

**注意:** 
1. 方式二存在内存泄漏问题,因为创建的client没有在程序结束后close掉.
2. 当用户名或密码带有 ‘:’, ‘/’, ‘+’ 和 ‘@’ 这些字符时需要使用百分比编码,否则连接报错: `Username and password must be escaped according to RFC 3986, use urllib.parse.quote_plus().`

### 插入数据(insert)

In [34]:
import random
import pymongo

MONGODB = {
    "user": "arvin",
    "passwd": "arvin123",
    "host": "127.0.0.1:27017",
    "dbname": "testdb"
}


def connect_mongo(mongo_param):
    uri =  'mongodb://{}:{}@{}/{}'.format(quote_plus(mongo_param['user']),
                                          quote_plus(mongo_param['passwd']),
                                          mongo_param['host'],
                                          mongo_param['dbname'])
    try:
        client = pymongo.MongoClient(uri, connect=False)
        mongodb = client[mongo_param['dbname']]
        return mongodb
    except Exception as e:
        print(e)
        
db = connect_mongo(MONGODB)

for i in range(10):
    name = ['xiaoming', 'frank', 'tom', 'jack', 'bomb']
    age = [18, 20, 19, 21, 22, 26, 24]
    weight = [120, 150 , 142, 123, 126, 135, 134]
    doc = {'name': random.choice(name), 'age':random.choice(age), 'weight': random.choice(weight), 'sex': 'male'}
    db['info'].insert_one(doc)

# 插入一条数据会返回一个ObjectId对象,ObjectId('5af8f8c62b5eba0bb8fece6d')    
result = movie.insert_one(doc)



### 删除数据(delete)

In [33]:
import pymongo

MONGODB = {
    "user": "arvin",
    "passwd": "arvin123",
    "host": "127.0.0.1:27017",
    "dbname": "testdb"
}


def connect_mongo(mongo_param):
    uri =  'mongodb://{}:{}@{}/{}'.format(quote_plus(mongo_param['user']),
                                          quote_plus(mongo_param['passwd']),
                                          mongo_param['host'],
                                          mongo_param['dbname'])
    try:
        client = pymongo.MongoClient(uri, connect=False)
        mongodb = client[mongo_param['dbname']]
        return mongodb
    except Exception as e:
        print(e)
        
db = connect_mongo(MONGODB)

# 删除一条数据
db['res_score'].delete_one({})

# 删除全部数据
db['res_score'].delete_many({})


### 更新数据(update)

In [41]:
import pymongo

MONGODB = {
    "user": "arvin",
    "passwd": "arvin123",
    "host": "127.0.0.1:27017",
    "dbname": "testdb"
}


def connect_mongo(mongo_param):
    uri =  'mongodb://{}:{}@{}/{}'.format(quote_plus(mongo_param['user']),
                                          quote_plus(mongo_param['passwd']),
                                          mongo_param['host'],
                                          mongo_param['dbname'])
    try:
        client = pymongo.MongoClient(uri, connect=False)
        mongodb = client[mongo_param['dbname']]
        return mongodb
    except Exception as e:
        print(e)
        
db = connect_mongo(MONGODB)
movie = db['info']

# 更新一条数据,局部更新(更新某一个字段)
result = movie.update_one({'name': 'jack'}, {'$set': {'weight': 132}})

print(result.matched_count)
print(result.modified_count)

# 更新多条数据
result = movie.update_many({'name': 'frank'}, {'$set': {'age': 32}})

print(result.matched_count)
print(result.modified_count)

# 更新整个文档
movie.update_one({'_id': item.get('_id')}, {' 除了_id字段的整个doc'})

1
0
3
3


### 查询数据

#### 查询指定字段最新一条记录

In [ ]:
res = movie.find({}).sort('_id', -1).limit(1)
if res:
    pprint(res[0])

**注意: ** limit返回的是一个迭代器,并不是数据结果

#### 查询结果排序

In [ ]:
db.Account.find({}).sort("UserName")  --默认为升序
db.Account.find({}).sort("UserName",pymongo.ASCENDING)   --升序
db.Account.find({}).sort("UserName",pymongo.DESCENDING)  --降序
# 或者
db.Account.find({}).sort("UserName", 1)   --升序
db.Account.find({}).sort("UserName", -1)  --降序

#### 多列结果排序

In [ ]:
db.Account.find().sort([("UserName",pymongo.ASCENDING),("Email",pymongo.DESCENDING)])

#### 嵌套查询

In [ ]:
# 查询doc中key为response且response中key为status_code=302 记录的数目
db.getCollection('urls').find({'response.status_code': 302}).count()

#### 查询返回部分字段,不返回整个文档

查询返回部分字段,不返回整个文档可以节省流量,降低带宽

* 第一个参数为查询条件，空代表查询所有文档
* 第二个参数中 1 代表选取该字段的值, 0代表过滤该字段的值

如果需要输出的字段比较多，不想要某个字段，可以用排除字段的方法 


In [ ]:
# 返回所有含有title和type字段的值,并不是包含这些字段的所有文档
db.news.find( {}, {'title': 1,  'type': 1} )

# 查询返回
db.news.find( {}, {'title': 1,  'type': 0} )

db.inventory.find( { type: 'food' }, { type:0 })


# 不输出内容字段，其它字段都输出
db.news.find( {}, {content: 0 } )

### map-reduce

In [ ]:
from bson.code import Code

mapfunc = Code('''function(){emit(this.tel, {platform: this.platform,time:this.time})}''')
reducefunc = Code(
    '''function(key, values){var time = 0;var platform = "";values.forEach(function(doc){if(doc.time>time){time=doc.time;platform=doc.platform;}});return {platform: platform,time:time};}''')
finalfunc = Code('''function(key, val){val.tel = key;return val;}''')

movie.map_reduce(mapfunc, reducefunc, out="collection_name", query={
                 "tel": {"$regex": '^[0-9]+$'}, "time": {"$gte": 0}}, finalize=finalfunc)

In [56]:
import random
import pymongo

MONGODB = {
    "user": "arvin",
    "passwd": "arvin123",
    "host": "127.0.0.1:27017",
    "dbname": "testdb"
}


def connect_mongo(mongo_param):
    uri =  'mongodb://{}:{}@{}/{}'.format(quote_plus(mongo_param['user']),
                                          quote_plus(mongo_param['passwd']),
                                          mongo_param['host'],
                                          mongo_param['dbname'])
    try:
        client = pymongo.MongoClient(uri, connect=False)
        mongodb = client[mongo_param['dbname']]
        return mongodb
    except Exception as e:
        print(e)
        
db = connect_mongo(MONGODB)

doc = {'site': 'https://www.xxxxxxxxxx.com'}
res = db['score'].insert_one(doc)


try:
    for year in [2018, 2019, 2020]:
        for month in range(1, 13):
            for day in range(1, 30):
                date_str = '{}-{}-{}'.format(year, month, day)
                data = [random.choice(range(30, 99)) for i in range(4)]
                db['score'].update_one({'_id': res.inserted_id}, {'$set': {date_str: data}}, upsert=True)
except Exception as e:
    print(e)
                

# for i in range(10):
#     name = ['xiaoming', 'frank', 'tom', 'jack', 'bomb']
#     age = [18, 20, 19, 21, 22, 26, 24]
#     weight = [120, 150 , 142, 123, 126, 135, 134]
#     doc = {'name': random.choice(name), 'age':random.choice(age), 'weight': random.choice(weight), 'sex': 'male'}
#     db['info'].insert_one(doc)

# # 插入一条数据会返回一个ObjectId对象,ObjectId('5af8f8c62b5eba0bb8fece6d')    
# result = movie.insert_one(doc)


In [4]:
import pymongo

# 仅仅创建mongodb连接,但未验证,没有连接到具体的数据库,相当于打开一个shell
client = pymongo.MongoClient(host='192.168.1.141', port=27017)
# 创建数据库
db = pymongo.database.Database(client, 'goshawk_test')
# db.command("createUser", "Goshawk", pwd="yYZmrjJ8xauAe#t7", roles=["readWrite"])

# 创建管理员用户,密码和权限
# db.command("createUser", "admin", pwd="admin@123456.", roles=["root"])
# 创建普通用户,密码和读取权限
# db.command("createUser", "arvin", pwd="arvin@123456.", roles=["readWrite"])